# E-commerce Business Sales Analysis and Visualization
<img src="https://drive.google.com/uc?export=view&id=1YeekIMo5PH33ZEV1clEfmU-wDD0PhfZA" width="750"> 

<font size="5"> Outline</font>

This notebook provides a sales analysis for an e-commerce business. The data contains 537,966 sales records and 9 columns, including a product description, quanitity of items sold, unit price, date of sale and country.

The analysis answers several business questions, among which:
* What is the overall sales trend?
* Which is the best selling product in each country?
* How many new customers are there each month?
* When do customers make the most purchases?

Most of the data visualizations are done using Plotly express, which allows for a relatively easy implementation of interactive graphs. Seaborn is also used to create heatmaps, matplotlib is used to create subplots and format figures, and conditional formatting (such as background gradients) is applied to several dataframes to highlight values.


The dataset used in the present notebook is the cleaned version of the original online retail business dataset. The notebook containing the cleaning, which I have also published, can be found [<ins>here</ins>](https://www.kaggle.com/atanaskanev/e-commerce-business-data-cleaning).

In short, the cleaning process included:
* cleaning erroneous and missing data
* removing duplicated descriptions for the same stockcodes
* handling outliers

I have also created an SQL notebook which replicates this analysis and answers the same business questions using PostgreSQL. You can see my SQL notebook on my Github [<ins>here</ins>](https://github.com/atanaskanev/ecommerce_sales_analysis/blob/main/ecommerce_business.ipynb).

## Table of Contents
Click on any heading to jump straight to the content

[<font size="5">Importing Libraries and Data</font>](#section-eight)

[<font size="5">Sales Analysis and Visualization</font>](#section-nine)
* [What are the Sales Figures for Each Country?](#section-ten)
* [What is the Overall Sales Trend?](#section-eleven)
* [How Many Customers Purchased Products Each Month?<br> How Many New Customers were There Each Month?](#section-twelve)
* [What Time During the Day Do Customers Make the Most Purchases?](#section-thirteen)
* [Which is the Best Selling Product in Each Country?](#section-fourteen)
* [Which are the Most Successful Products Overall?](#section-fifteen)
* [Which Customers Contributed the Most to Total Sales?](#section-sixteen)

<a id="section-eight"></a>
# Importing Libraries and Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # creating subplots and formating figures
import seaborn as sns # visualizations including heatmaps
import plotly.express as px # used for interactive visualizations
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot # plot plotly graphs in line in a notebook
init_notebook_mode(connected = True)
import calendar # used to convert numbers between 1 and 12 to month names

import warnings        
warnings.filterwarnings("ignore") # ignores warnings

In [ ]:
# importing the cleaned dataset
data = pd.read_csv("../input/online-retail-business-cleaned-dataset/online_retail_cleaned.csv")

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
# cast InvoiceDate as a date type
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])

In [ ]:
data.isna().sum()

In [ ]:
data.head(10)

<a id="section-nine"></a>
# Sales Analysis and Visualization

<a id="section-ten"></a>
## What are the Sales Figures for Each Country?
First, let's see what percentage of total sales each country accounts for:

In [ ]:
# calculate total sales by country
country_sales = pd.DataFrame(data.groupby("Country") \
["ItemTotal"].sum()).reset_index().rename({"ItemTotal":"TotalSales"},axis=1)

In [ ]:
fig = px.pie(country_sales,
             values="TotalSales",
             names="Country",
             title="Percent of Total Sales by Country",
             color_discrete_sequence=px.colors.qualitative.G10
            )
# px.colors.qualitative.swatches().show() # see available color palettes

fig.update_traces(
                  textposition="inside",
                  textinfo="percent+label"
                 )
fig.update_layout(
                  margin=dict(l=10, r=50, b=10, t=70, pad=0),
                  titlefont = dict(size = 20)
                 )
iplot(fig)

We see that the UK accounts for the vast majority of total sales. 

Let's visualize other countries' share of total sales by plotting the sales figure on a map. We will plot this map without the UK sales since it would skew the scale due to its predominant share of sales.

In [ ]:
# also drop "Unspecified" and "European Community" since they cannot be mapped
country_sales = country_sales[country_sales["Country"] \
.isin(["Unspecified", "European Community","United Kingdom"]) == False]

In [ ]:
# rename countries into names which are recognised by plotly
country_sales["Country"] = \
country_sales["Country"].replace({"EIRE": "Ireland",
                                  "Channel Islands": "United Kingdom",
                                  "RSA": "South Africa"})

Note that the following map is interactive so you can zoom in and out, and can hover on different countries to show sales figures.

In [ ]:
# https://stackoverflow.com/questions/59812824/plotly-express-plot-not-shown-in-jupyter-notebook
choro_data = dict(
                  type = "choropleth",
                  colorscale = "Agsunset",
                  locations = country_sales["Country"],
                  locationmode = "country names",
                  z = country_sales["TotalSales"].astype(float).round(0).tolist(),
                  hovertext = country_sales["Country"],
                  hovertemplate = "%{hovertext}: £%{z:,.0f} <extra></extra>",
                  # https://stackoverflow.com/questions/59057881/python-plotly-how-to-customize-hover-template-on-with-what-information-to-show
                  colorbar = {"title" : "Total Sales", },
                ) 

layout = dict(
              title = "Total Sales By Country Excluding the UK",
              titlefont = dict(size = 20),
              geo = dict(showframe = False, bgcolor = "#BAEAED",),
              margin={"r":0,"t":50,"l":10,"b":0}  
             )

fig = go.Figure(data=choro_data, layout=layout)
# fig.update_traces(visible = True)
iplot(fig)


We see that apart from the UK, Australia and countries in Europe have contributed the most towards total sales.

Let's zoom in on Europe:

In [ ]:
choro_data = [dict(
                  type = "choropleth",
                  colorscale = "Agsunset",
                  locations = country_sales["Country"],
                  locationmode = "country names",
                  z = country_sales["TotalSales"].astype(float).round(0).tolist(),
                  hovertext = country_sales["Country"],
                  hovertemplate = "%{hovertext}: £%{z:,.0f} <extra></extra>",
                  # https://stackoverflow.com/questions/59057881/python-plotly-how-to-customize-hover-template-on-with-what-information-to-show
                  colorbar = {"title" : "Total Sales", },
                 ),
              dict(
                  type = "scattergeo", # add static country labels
                  locations = country_sales["Country"],
                  locationmode = "country names",
                  text = country_sales["Country"],
                  hoverinfo = "skip", # do not display static label when hovering over the country
                  mode = "text",
                  textfont=dict( # format the static country labels
                                size = 11, # only these 3 properties are allowed
                                color = "white",
                                family = "Arial"
                               )
                  ) 
            ]
layout = dict(
              title = "Total Sales By Country Excluding the UK",
              titlefont = dict(size = 20),
              geo = dict(
                         showframe = False,
                         bgcolor = "#A4E0E4",
                         projection = dict(scale = 4.5), # default zoom
                         center = dict(lat = 47, lon = 5) # default map position
                         ),
              margin={"r":0,"t":50,"l":10,"b":0}
             )

fig = go.Figure(data = choro_data, layout = layout)
iplot(fig)


<a id="section-eleven"></a>
## What is the Overall Sales Trend?

Check date range:

In [ ]:
print("first date: ", data["InvoiceDate"].min(),"\n", "last date: ", data["InvoiceDate"].max())

We see the data spans between 12 January 2010 and 10 December 2011, which is almost 2 years. 

Note that since the data does not cover the whole month of December 2011, sales figures are likely going to be low in this month, since they are about 12 days only. 

In [ ]:
# create columns extracting the year and month of InvoiceDates
data["year"], data["month"] = data["InvoiceDate"].dt.year, data["InvoiceDate"].dt.month

In [ ]:
sales = data.groupby(["year","month"])["ItemTotal"].sum() \
.reset_index().rename({"ItemTotal":"TotalSales"},axis=1)

sales

It turns out we have no data about April 2010 and November 2010, so we add two columns with the averages of the preceding and following months for consistency. The averages were calculated, and instead of using a formula, the actual values are inserted for brevity.

In [ ]:
# create rows about April 2010 and November 2010 with averages from preceding and following month 
                                                                                                  
new_rows = pd.DataFrame({"year":[2010,2010],
                         "month": [4,11],
                         "TotalSales": [38123.21,184980.04]},
                         index = [98,99]) # arbitrary indexes

# insert the row in the sales table
sales = pd.concat([new_rows, sales]) \
.sort_values(by=["year","month"]).reset_index(drop=True)

Some formatting:

In [ ]:
# convert numbers into month names
sales["month"] = sales["month"].apply(lambda x: calendar.month_abbr[x])

# combine month and year
sales["month"] = sales["month"].astype(str) + " " + sales["year"].astype(str)

# drop the redundant year column
sales = sales.drop("year", axis = 1) 

sales = sales[0:23] # drop December 2011 since the data does not cover the whole month

And we visualize. The graph is interactive so you can zoom in/out and hover over the data to see details.

In [ ]:
# line chart using plotly expess Scatter
trace = go.Scatter(
                    x = sales["month"],
                    y = sales["TotalSales"],
                    mode = "lines+markers",
                    name = "TotalSales",
                    line = dict(width = 4),
                    marker = dict(
                                  size = 10,
                                  color = "rgba(120, 26, 120, 0.8)"
                                 ),
                    hovertemplate = " %{x}<br>£%{y:,.0f} <extra></extra>",
                  )
line_data = [trace]
layout = dict(
              title = "Total Sales by Month",
              titlefont = dict(size = 20),
              margin=dict(l=10, r=50, b=10, t=70, pad=0),
              xaxis= dict(title= "Month",ticklen = 5,zeroline = False),
              yaxis= dict(title= "Total Sales", tickformat = ",.0f", tickprefix="£")
             )
fig = dict(data = line_data, layout = layout)
iplot(fig)

We see a clear upward trend with initial sharp increses towards the end of 2010 and the beginning of 2011. Let's investigate whether the number of customers in each month exhibit a similar trend.

<a id="section-twelve"></a>
##  How Many Customers Purchased Products Each Month? <br> How Many New Customers were There Each Month?
We saw some records with missing CustomerIDs above, so first we check how many unique Invoices there are and how many of them have a missing CustomerID:

In [ ]:
# all orders
len(data["InvoiceNo"].unique())

In [ ]:
# invoices with at least one record with missing CustomerID
len(data[data["CustomerID"].isna()]["InvoiceNo"].unique())

We can also check how many invoices have <ins>all</ins> of their CustomerIDs missing.

In [ ]:
data.groupby("InvoiceNo").apply(lambda x: all(np.isnan(i) for i in x["CustomerID"])).tolist().count(True)

As we see, the number is the same, which means that invoices either have missing values for all CustomerIDs, or no CustomerIDs are missing.

Since we do not know how many customers have an NaN value for CustomerID, we drop records with missing CustomerIDs for this analysis.

In [ ]:
customers = data[data["CustomerID"].notna()].groupby(["year", "month"]) \
.agg({"CustomerID": "unique"}) \
.reset_index().rename({"CustomerID": "unique_customer_ids"}, axis = 1)

# calculate the number of unique customers and insert it as a column
customers.insert(2,"unique_customers_this_month", customers["unique_customer_ids"].str.len())

customers.head()


New customers are customers who purchase an item for the first time. In other words, these are customers whose CustomerID appears in the records for the first time at the date of the purchase.

To find which CustomerIDs appear for the first time, we create a running list which accumulates all unique_customer_ids up to each month. Then we remove duplicates and check the length of the list for each month. In doing so, we get the running total of unique customers.

In [ ]:
ids = []

# creates a running list of customerids up to each month
for index, row in customers.iterrows(): 
    if index == 0:
        ids.append(row["unique_customer_ids"].tolist())
        
    else:   # adds the present ids to the accumulated list of previous ids
        ids.append(row["unique_customer_ids"].tolist() + ids[index-1])

In [ ]:
total_customers = []
for i in range(len(ids)):
    total_customers.append(len(set(ids[i]))) # the set removes duplicates  
    
# insert as a column
customers.insert(3, "total_customers", total_customers)

In [ ]:
# add the first difference of totaL_customers

customers.insert(3, "new_customers_this_month", customers["total_customers"].diff() \
.replace({np.nan: 98}).astype(int)) # fill in the first value

In [ ]:
 # drop the long lists of unique customers
customers = customers.drop("unique_customer_ids", axis = 1)

In [ ]:
# create rows about April 2010 and November 2010 with averages from preceding and following months
new_rows = \
pd.DataFrame({"year":[2010,2010],
              "month": [4,11],
              "unique_customers_this_month": [65,271],
              "new_customers_this_month": [59,163],
              "total_customers": [288,803]}, index = [98,99]) # arbitrary indexes

# insert the row in the customers table
customers = pd.concat([new_rows, customers]) \
.sort_values(by=["year","month"]).reset_index(drop=True)

Some formatting:

In [ ]:
# convert numbers into month names
customers["month"] = customers["month"].apply(lambda x: calendar.month_abbr[x])

# combine month and year
customers["month"] = customers["month"].astype(str) + " " + customers["year"].astype(str)

# drop the redundant year column
customers = customers.drop("year", axis = 1)

customers = customers[0:23] # drop December 2011 since the data does not cover the whole month

And plot:

In [ ]:
trace1 = go.Scatter(
                    x = customers["month"],
                    y = customers["unique_customers_this_month"],
                    mode = "lines+markers",
                    name = "Unique Customers This Month",
                    line = dict(width = 4),
                    marker = dict(
                                  size = 10,
                                  color = "#0E79B2"
                                 ),
                    hovertemplate = "%{x}<br>Unique Customers: %{y} <extra></extra>",
                  )
trace2 = go.Scatter(
                    x = customers["month"],
                    y = customers["new_customers_this_month"],
                    mode = "lines+markers",
                    name = "New Customers This Month",
                    line = dict(width = 4),
                    marker = dict(
                                  size = 10,
                                  color = "rgba(242, 225, 39, 1)"
                                 ),
                    hovertemplate = "%{x}<br>New Customers: %{y} <extra></extra>",
                  )

line_data = [trace1, trace2]

layout = dict(
              title = "Customers by Month",
              titlefont = dict(size = 20),
              margin=dict(l=10, r=50, b=10, t=70, pad=0),
              xaxis= dict(title= "Month",ticklen = 5,zeroline = False),
              yaxis= dict(title= "Number of Customers"),
              legend=dict(
                          font = dict(size = 12),
                          yanchor = "top",
                          y=0.98,
                          x= 0.01
                         )
             )
fig = dict(data = line_data, layout = layout)
iplot(fig)

Wee see a significant jump in new customers around December 2010 and January 2011, likely related to the Christmas holidays. We could investigate whether the business has been running marketing campaigns around these times, and which campaigns were the most successful.

We can see that after the spike of new customers around December/January, the number of customers per month has been increasing steadity. We can see another spike in customers in November 2011, although there is no spike in new customers. This might suggest that customers were satisfied with their Christmas purchases in 2010, and are now coming back to the online store for their Christmas 2011 purchases.

We do see that the number of new customers has been decreasing slowly over the year. This could once again be related to marketing campaigns and could be investigated.


Let's look at the chart with total customers added:

In [ ]:
trace1 = go.Scatter(
                    x = customers["month"],
                    y = customers["unique_customers_this_month"],
                    mode = "lines+markers",
                    name = "Unique Customers This Month",
                    line = dict(width = 4),
                    marker = dict(
                                  size = 10,
                                  color = "#0E79B2"
                                 ),
                    hovertemplate = "%{x}<br>Unique Customers: %{y} <extra></extra>",
                  )
trace2 = go.Scatter(
                    x = customers["month"],
                    y = customers["new_customers_this_month"],
                    mode = "lines+markers",
                    name = "New Customers This Month",
                    line = dict(width = 4),
                    marker = dict(
                                  size = 10,
                                  color = "rgba(242, 225, 39, 1)"
                                 ),
                    hovertemplate = "%{x}<br>New Customers: %{y} <extra></extra>",
                  )
trace3 = go.Scatter(
                    x = customers["month"],
                    y = customers["total_customers"],
                    mode = "lines+markers",
                    name = "Total Customers",
                    line = dict(width = 4),
                    marker = dict(
                                  size = 10,
                                  color = "rgba(242, 39, 127, 1)"
                                 ),
                    hovertemplate = "%{x}<br>Total Customers: %{y} <extra></extra>",
                  )

line_data = [trace1, trace2, trace3]

layout = dict(
              title = "Customers by Month",
              titlefont = dict(size = 20),
              margin=dict(l=10, r=50, b=10, t=70, pad=0),
              xaxis= dict(title= "Month",ticklen = 5,zeroline = False),
              yaxis= dict(title= "Number of Customers"),
              legend=dict(
                          font = dict(size = 12),
                          yanchor = "top",
                          y=0.98,
                          x= 0.01
                         )
             )
fig = dict(data = line_data, layout = layout)
iplot(fig)

We do infact see that the line chart of the number of total customers has a concave shape. In other words, the number of total customers is increasing at a descreasing rate. As commented above, this is directly related to the decreasing number of new customers.

<a id="section-thirteen"></a>
## What Time During the Day Do Customers Make the Most Purchases? 
We have seen spikes in sales around December 2010, which is likely related to the Christmas holidays. Since the last date of this dataset is December 2011 (which we assume is the present), we expect that the business would be running marketing campaigns for this year's Christmas hodidays as well. Therefore, it would be useful to find out what time of day customers make the most purchases in order to target the marketing campaigns around these times.

To check this, we can look at what time customers were making the most orders the previous December (i.e. December 2010), as well as the most common hours for purchases during the last 2 months (i.e. October and November 2011). Since we have data about 12 days of December 2011, we include these in the analysis as well.

In [ ]:
# take data only about December 2010 and October, Novermber and December 2011
subset = data[
              ((data["year"] == 2010) & (data["month"] == 12))
              |    
              ((data["year"] == 2011) & (data["month"] == 10))
              |
              ((data["year"] == 2011) & (data["month"] == 11))
              |
              ((data["year"] == 2011) & (data["month"] == 12))
              ]

In [ ]:
# extract the hour of purchase from InvoiceDate and add it as a column
subset["hour"] = subset["InvoiceDate"].astype(str).str[11:13].astype(int)

In [ ]:
# calculate the total number of orders for each hour of the day in these months

frequency = subset.groupby(["year","month","hour"]) \
.agg({"InvoiceNo":"nunique"}).reset_index() \
.rename({"InvoiceNo": "num_orders"}, axis = 1)

frequency.head()

We pivot the table, so it is easier to read and easier to plot:

In [ ]:
pivot = frequency.pivot(index = "hour", columns = ["year","month"], values = ["num_orders"])

pivot = pd.DataFrame(pivot.to_records()) # flattens multilevel column headings

pivot["hour"] = pivot["hour"].astype(str) + ":00" # make hours more readable

pivot = pivot.set_index("hour")

pivot.index.name = "" # remove index name for plotting

pivot.rename(columns={ # set more readable names
                      pivot.columns[0]:"Dec 2010",
                      pivot.columns[1]:"Oct 2011",
                      pivot.columns[2]:"Nov 2011",
                      pivot.columns[3]:"Dec 2011"
                      }, inplace = True)

pivot

And we plot a heatmap of these values for each month:

In [ ]:
fig, axs = plt.subplots(4,1, figsize = (9,2.7))
cbar_ax = fig.add_axes([0.92, .10, .03, .8]) # add custom colorbar

sns.heatmap(pd.DataFrame(pivot["Dec 2010"]).T, ax = axs[0], cbar = False)
sns.heatmap(pd.DataFrame(pivot["Oct 2011"]).T, ax = axs[1], cbar_ax = cbar_ax, xticklabels=False)
sns.heatmap(pd.DataFrame(pivot["Nov 2011"]).T, ax = axs[2], cbar = False, xticklabels=False)
sns.heatmap(pd.DataFrame(pivot["Dec 2011"]).T, ax = axs[3], cbar = False, xticklabels=False)

cbar = axs[1].collections[0].colorbar # set custom colorbar labels
cbar.set_ticks([30, 190, 350])
cbar.set_ticklabels(["low", "medium", "high"])

axs[0].xaxis.set_ticks_position("top")
axs[0].yaxis.set_ticklabels(axs[0].get_yticklabels(), rotation = 0)
axs[1].yaxis.set_ticklabels(axs[1].get_yticklabels(), rotation = 0)
axs[2].yaxis.set_ticklabels(axs[2].get_yticklabels(), rotation = 0)
axs[3].yaxis.set_ticklabels(axs[3].get_yticklabels(), rotation = 0)

fig.suptitle("Total Number of Orders for Each Hour of the Day", y = 1.10)
plt.show()

Or alternatively, we could have applied background gradients directly to the dataframe:

In [ ]:
pivot.replace(np.nan,0).style\
.background_gradient(cmap="rocket", subset=["Dec 2010"])\
.background_gradient(cmap="rocket", subset=["Oct 2011"])\
.background_gradient(cmap="rocket", subset=["Nov 2011"])\
.background_gradient(cmap="rocket", subset=["Dec 2011"])\
.format("{:.0f}")

From these figures, it is clear that the most orders are placed around midday, more specifically at 12:00. Therefore, the marketing team could target their campaigns around these times to maximize conversions.

<a id="section-fourteen"></a>
## Which is the Best Selling Product in Each Country?
We are going to investigate which product generated the highest sales in each country, how much sales it generated, and what percent of the country's total sales it generated. We also find which product is the most common best seller.

In [ ]:
# calculate total sales for each product in each country
sales_countr_descr = data.groupby(["Country", "Description"]) \
.agg({"ItemTotal": "sum"}) \
.rename({"ItemTotal":"TotalSales"},axis=1).reset_index()

sales_countr_descr

In [ ]:
# find the total sales of the best selling product in each country
max_sales = pd.DataFrame(sales_countr_descr.groupby("Country") \
["TotalSales"].max().reset_index()) \
.rename({"TotalSales":"Best_Product_Total_Sales"},axis=1)

max_sales.head()

In [ ]:
# join the two tables from above together
joined_df = sales_countr_descr.merge(max_sales, on = "Country", how = "left")
joined_df

In [ ]:
# find the products whose sales equal the highest product sales in each country
# in other words, find the best selling item in each country
joined_df = joined_df[joined_df["TotalSales"] == joined_df["Best_Product_Total_Sales"]]
joined_df.head()

In [ ]:
# drop the redundant column
joined_df = joined_df.drop("Best_Product_Total_Sales", axis = 1)

Now we want to find the percentage of total sales these products have generated in each country. To do this we create a table with the countries' total sales and join it to the table above:

In [ ]:
# total sales in each country
country_sales = pd.DataFrame(data.groupby("Country")["ItemTotal"].sum()) \
.reset_index().rename({"ItemTotal":"Country_Total_Sales"}, axis = 1)

country_sales.head()

In [ ]:
joined_df = joined_df.merge(country_sales, on = "Country", how = "inner") \
.rename({"Description":"Best_Selling_Product"}, axis = 1)

joined_df.head()

And add another column calculating the percentage of total sales which the Best selling items has generated in each country. It is formatted as a percentage with 2 decimal points. A background gradient is applied.

In [ ]:
joined_df["%_of_Country_Sales"] = (joined_df["TotalSales"] / joined_df["Country_Total_Sales"])

joined_df \
.style.background_gradient(cmap=sns.light_palette("seagreen", as_cmap=True), \
 subset=["%_of_Country_Sales"]) \
.format({"%_of_Country_Sales":"{:.2%}",
         "TotalSales":"£{:,.0f}",
         "Country_Total_Sales":"£{:,.0f}"})

Which product is the most common best seller? 

In [ ]:
pd.DataFrame(joined_df["Best_Selling_Product"].value_counts()) \
.rename({"Best_Selling_Product":"Best_Selling_Product_in_X_Countries"},axis=1)

<a id="section-fifteen"></a>
## Which are the Most Successful Products Overall?
A characteristic we could use to answer this is: which products of relatively high price have sold relatively high quantities. This can be easily visualized with a scatter plot of all products with total quantity sold and average unitprice on the x and y axes.

Products' markers to the north-east on the scatter plot suggest relatively high quantities sold at a relatively high price.  

In [ ]:
# find total quantity sold and average unit price for all products
products = data.groupby("Description") \
.agg({"Quantity":"sum", "UnitPrice":"mean"}).reset_index()

products = products[products["Description"].isin(["DOTCOM","Manual","Discount"]) == False] # remove for plotting

products

The following scatter plot is interactive, so you can hover over any point to see its details.

In [ ]:
trace = go.Scatter(
                    x = products["Quantity"],
                    y = products["UnitPrice"],
                    customdata = products["Description"],
                    hovertemplate = "%{customdata}<br>Quantity Sold: %{x}<br>UnitPrice: £%{y:.2f}<br><extra></extra>",
                    mode = "markers",
                    name = "Products",
                    line = dict(width = 4),
                    marker = dict(
                                  size = 10,
                                  color = "#1199cf"
                                 )
                    )

scatter_data = [trace]

layout = dict(
              title = "Products",
              titlefont = dict(size = 20),
              margin=dict(l=10, r=50, b=10, t=70, pad=0),
              xaxis= dict(title= "Quantity Sold",ticklen = 5,zeroline = False),
              yaxis= dict(title= "Unit Price")
             )
fig = dict(data = scatter_data, layout = layout)
iplot(fig)

Above we saw "REGENCY CAKESTAND 3 TIER" as the most common best seller (8 countries) and here we see it as a high selling product as well - its marker is visually separated from the other products on the plot. 

<a id="section-sixteen"></a>
## Which Customers Contributed the Most to Total Sales?
Finally, let's see the customers with highest purchase figures and their corresponding number of orders: 

In [ ]:
cust_purchases = \
data.groupby("CustomerID") \
.agg({"InvoiceNo": "nunique", "ItemTotal": "sum"}) \
.rename({"ItemTotal":"TotalPurchases","InvoiceNo":"Number_of_Orders"},axis = 1) \
.sort_values(by = "TotalPurchases", ascending = False)

cust_purchases.head(15) # only first 15

How much did each customer contribute to the total sales figure in percentage terms?

In [ ]:
cust_purchases["Percent_of_TotalSales"] = (cust_purchases["TotalPurchases"]/data["ItemTotal"].sum()).map("{:.2%}".format)
cust_purchases["TotalPurchases"] = cust_purchases["TotalPurchases"].map("£{:,.0f}".format) # format as currency

cust_purchases.head(15) # first 15 only


<font size="5">Thank you for reading my notebook!</font>

I have also created an SQL notebook which replicates this analysis and answers the same business questions using PostgreSQL. You can see my SQL notebook on my Github [<ins>here</ins>](https://github.com/atanaskanev/ecommerce_sales_analysis/blob/main/ecommerce_business.ipynb).

Any comments and suggestions are highly appreciated!